In [1]:
import pandas as pd
import os
import random

# Input file and parameters
pairs_file = "/home/bjangley/VPR/mast3r-v2/results/spagna_matches_inliers_fm_top3_anchors_per_query_per_anchorseq.csv"
pairs_df = pd.read_csv(pairs_file)
min_inliers = 650

# Filter rows based on the minimum number of inliers
filtered_rows = pairs_df[pairs_df['num_inliers'] > min_inliers]

# Calculate the number of pairs lost due to filtering
print(f"Total pairs before filtering: {len(pairs_df)}")
print(f"Total pairs after filtering: {len(filtered_rows)}")


# Extract relevant columns
anchor_idxs = filtered_rows['anchor_idx'].astype(int).tolist()
query_idxs = filtered_rows['query_idx'].astype(int).tolist()
anchor_seqs = filtered_rows['anchor_seq'].astype(int).tolist()
query_seqs = filtered_rows['query_seq'].astype(int).tolist()

# Combine anchor-query pairs into a list
# pairs = [(anchor_seqs[idx], anchor_idxs[idx], query_seqs[idx], query_idxs[idx]) for idx in range(len(anchor_idxs))]
pairs = [(anchor_idxs[idx], query_idxs[idx]) for idx in range(len(anchor_idxs))]

# Shuffle the pairs randomly
random.seed(42)  # Set seed for reproducibility
random.shuffle(pairs)

# Define split proportions
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate split indices
total_pairs = len(pairs)
train_end = int(total_pairs * train_ratio)
val_end = train_end + int(total_pairs * val_ratio)

# Split the pairs
train_pairs = pairs[:train_end]
val_pairs = pairs[train_end:val_end]
test_pairs = pairs[val_end:]
print(len(train_pairs), len(val_pairs), len(val_pairs))

Total pairs before filtering: 1347
Total pairs after filtering: 566
396 84 84


In [3]:
# Output file paths
pairs_path = "/home/bjangley/VPR/mast3r-v2/pairsVBR/spagna"
train_output = os.path.join(pairs_path, "spagna_train_pairs.txt")
val_output = os.path.join(pairs_path, "spagna_val_pairs.txt")
test_output = os.path.join(pairs_path, "spagna_test_pairs.txt")

# Save the splits into separate files
def save_pairs_to_file(pairs, output_file):
    with open(output_file, "w") as f:
        for pair in pairs:
            # Write anchor and query pair in the format: anchor_seq, anchor_idx, query_seq, query_idx
            f.write(f"{pair[0]} {pair[1]}\n")

save_pairs_to_file(train_pairs, train_output)
save_pairs_to_file(val_pairs, val_output)
save_pairs_to_file(test_pairs, test_output)

print(f"Train pairs saved to {train_output}")
print(f"Validation pairs saved to {val_output}")
print(f"Test pairs saved to {test_output}")

Train pairs saved to /home/bjangley/VPR/mast3r-v2/pairsVBR/spagna/spagna_train_pairs.txt
Validation pairs saved to /home/bjangley/VPR/mast3r-v2/pairsVBR/spagna/spagna_val_pairs.txt
Test pairs saved to /home/bjangley/VPR/mast3r-v2/pairsVBR/spagna/spagna_test_pairs.txt


In [23]:
print(len(query_idxs))
print(len(anchor_idxs))


print("train", train_pairs[0:10])

566
566
train [(1100, 20550), (8600, 10400), (41950, 5500), (1350, 20750), (1250, 20700), (9900, 22250), (9000, 21350), (1500, 6900), (9650, 7850), (1300, 6700)]


In [4]:

from my_vbr_utils.vbr_dataset import VBRDataset, load_scene_calibration
from my_vbr_utils.utilities import load_scene_correspondences

# --- Config and Dataset Loading ---
# Set these variables to configure your scene and utility path
scene_name = 'spagna_train0'
vbr_utils_root = '/home/bjangley/VPR/mast3r-v2/my_vbr_utils'

config_path = f'{vbr_utils_root}/vbrPaths.yaml'

# Load dataset and calibration
all_loaded = VBRDataset(config_path, scenes=[scene_name])
vbr_scene = all_loaded.get_combined_dataset()
calib = load_scene_calibration(scene_name, config_path)

Loading scene: spagna_train0
  KITTI path: /datasets/vbr_slam/spagna/spagna_train0_kitti
  GT path: /datasets/vbr_slam/spagna/spagna_train0/spagna_train0_gt.txt
  ✓ Successfully loaded spagna_train0
Loaded 1 training scenes: ['spagna_train0']


In [5]:
from PIL import Image
from my_utils.my_vbr_dataset import generate_depth_and_scene_maps
import numpy as np
OUTPUT_DIR="/home/bjangley/VPR/vbr_depthmaps"
T_cam_lidar = calib['cam_l']["T_cam_lidar"]
K = calib['cam_l']['K']
print(T_cam_lidar, K)
for idx in range(len(pairs)):
    pair = pairs[idx]
    for image in pair:
        item = vbr_scene[image]
        img_path = item['image']
        lidar_pts = item['lidar_points']
        if lidar_pts.shape[0] < 5:
            print(f"[{image}] Skipped (no lidar)")
            continue
        # Load image to get size
        img = Image.open(img_path)
        img_shape = img.size[::-1]  # (H, W)
        # print(img_shape)
        # Generate depth and scene maps
        depth, scene = generate_depth_and_scene_maps(lidar_pts, K, T_cam_lidar, img_shape)
        # Save .npy
        out_path = os.path.join(OUTPUT_DIR, f"{image:010d}.npy")
        np.save(out_path, depth.astype(np.float32))
        # print(f"[{image}] Saved to {out_path}")

[[ 0.00561709 -0.9999834   0.00132127  0.23482969]
 [-0.0012881  -0.00132847 -0.99999831 -0.66564557]
 [ 0.9999835   0.0056154  -0.00129556 -0.07291627]
 [ 0.          0.          0.          1.        ]] [[1.26541574e+03 0.00000000e+00 6.49176487e+02]
 [0.00000000e+00 1.25912503e+03 3.49118750e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(700, 1388)
(7

In [6]:

# Count the number of .npy files in the output directory
npy_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.npy')]
num_npy_files = len(npy_files)

# Extract unique anchors and queries from the pairs
anchors = set(pair[0] for pair in pairs)
queries = set(pair[1] for pair in pairs)

num_unique_anchors = len(anchors)
num_unique_queries = len(queries)

# Print the results
print(f"Number of .npy files in {OUTPUT_DIR}: {num_npy_files}")
print(f"Number of unique anchors: {num_unique_anchors}")
print(f"Number of unique queries: {num_unique_queries}")

Number of .npy files in /home/bjangley/VPR/vbr_depthmaps: 311
Number of unique anchors: 115
Number of unique queries: 196


In [25]:
import os
from my_utils.transformations import pose_to_se3, se3_to_pose

# Output file path
poses_output_file = os.path.join(OUTPUT_DIR, "spagna_gt.txt")

# Open the file for writing
with open(poses_output_file, "w") as f:
    for idx in range(len(vbr_scene)):
        # Get the pose for the current image
        pose = vbr_scene.get_pose(idx)
        # Transform the pose to the camera frame
        # pose_cam = se3_to_pose(T_cam_lidar @ pose_to_se3(pose))
        # Write the pose to the file
        f.write(f"{pose[0]:.6f} {pose[1]:.6f} {pose[2]:.6f} {pose[3]:.6f} {pose[4]:.6f} {pose[5]:.6f} {pose[6]:.6f}\n")

print(f"Ground truth poses saved to {poses_output_file}")
    

Ground truth poses saved to /home/bjangley/VPR/vbr_depthmaps/spagna_gt.txt


In [24]:

pose = vbr_scene.get_pose(1)
pose = se3_to_pose(T_cam_lidar @ pose_to_se3(pose))
print(T_cam_lidar)
# Print the pose in a readable format with decimals
print("Pose (Readable Format):")
print(f"x: {pose[0]:.6f}, y: {pose[1]:.6f}, z: {pose[2]:.6f}, "
      f"qx: {pose[3]:.6f}, qy: {pose[4]:.6f}, qz: {pose[5]:.6f}, qw: {pose[6]:.6f}")

[[ 0.00561709 -0.9999834   0.00132127  0.23482969]
 [-0.0012881  -0.00132847 -0.99999831 -0.66564557]
 [ 0.9999835   0.0056154  -0.00129556 -0.07291627]
 [ 0.          0.          0.          1.        ]]
Pose (Readable Format):
x: 0.236370, y: -0.666579, z: -0.073709, qx: 0.502584, qy: -0.498216, qz: 0.498269, qw: 0.500918
